# USEFUL FUNCTIONS, TEST SITE

# FUNCTION 1:  description2vector(desc)

In [5]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

#### Functions:

In [23]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text


def sanitize_text(df, text_field, new_text_field_name):
    stop_words = set(stopwords.words('english')) 

    # CLEAN
    df[new_text_field_name] = df[text_field].str.lower()

    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem)) 

    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))

    # DISINFECT - aka TOKENIZATION
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: word_tokenize(elem))

    # Remove STOP words
    for ii in df.index:
        try:
            currii=ii
            #df.loc[ii,new_text_field_name] = [w for w in df.loc[ii,new_text_field_name] if not w in stop_words]
        except:
            print('this ii: {}'.format(ii))
        
    # CAUTERIZE - aka LEMMATIZATION
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: word_lemmatizer(elem))
    
    return df

def desc2docvect(currdoc,words):
    ''' Get the document vector from a description'''
    gamedocvects_df = pd.DataFrame({})
    numw = len(currdoc)
    docvect_df = pd.DataFrame({})
    for w in range(0,numw): # Over all words in the doc
        if currdoc[w] in words:
            docvect_df = pd.concat((docvect_df,pd.DataFrame(bgg_model.wv[currdoc[w]])),axis=1)
    gamedocvects_df = pd.concat((gamedocvects_df,docvect_df.mean(axis=1)),axis=1)
    return gamedocvects_df

#### INITIALIZE

In [24]:
# Get the word2vec model
bgg_model = Word2Vec.load('datasources/bgg_w2v_model.bin')
words = list(bgg_model.wv.vocab)

# Stop words
stop_words = set(stopwords.words('english')) 



#### EXTRACT DATA AND RETURN

In [ ]:
dv = desc2docvect(desc,words)

#### test:

In [25]:
desc = 'my SO and I recently started our venture into board gaming, and it quite quickly became apparent that we are very different players: she likes the solitaire euro games (e.g. she loves Terraforming Mars and Ticket to Ride), where she can ponder best move (talk about analysis paralysis) and ( literally almost always) come out on top. I instead am more impatient, get fairly quickly bored with the aforementioned games, and prefer games where there is tons of player interaction and a strong theme (e.g. ArkhM LCG, which she hates 😂). We have settled on more abstract simpler games like Azul and coop games - pandemic (looking forward to season 1 arriving soon) and considering spirit island. '

In [26]:
desc

'my SO and I recently started our venture into board gaming, and it quite quickly became apparent that we are very different players: she likes the solitaire euro games (e.g. she loves Terraforming Mars and Ticket to Ride), where she can ponder best move (talk about analysis paralysis) and ( literally almost always) come out on top. I instead am more impatient, get fairly quickly bored with the aforementioned games, and prefer games where there is tons of player interaction and a strong theme (e.g. ArkhM LCG, which she hates 😂). We have settled on more abstract simpler games like Azul and coop games - pandemic (looking forward to season 1 arriving soon) and considering spirit island. '

In [27]:
dv = desc2docvect(desc,words)

In [28]:
dv

,0
0,0.103770
1,-0.294371
2,-0.301576
3,0.008635
4,-0.118068
...,...
95,0.151769
96,0.101351
97,-0.012899
98,-0.171627


# %%%%%%%%%%%%%%%%%%%%%%%%%%%

# %%%%%%%%%%%%%%%%%%%%%%%%%%%

# FUNCTION 2:  vector2similargameurls(docvect)

In [ ]:
def vector2similargameurls(docvect):
    import pandas as pd
    import numpy as np
    import sklearn.metrics.pairwise as sklpw
    
    ##### GET THE DATA
    # Game data
    allgamedata_df = pd.read_pickle('datasources/BGG_FINAL.pkl') # USE ONLY FOR URLS
    allgamedata_df = allgamedata_df.astype({'game_rank':'int32'},copy=True)
    finalgamelist_df = pd.read_pickle('datasources/BGG_GameSimilarityKey.pkl')
    finalgamelist_df.reset_index(drop=True,inplace=True) # So that row ids are indices to gamevector array
    # Semantic game vectors
    allgamedocvects = np.load('datasources/allgamedocvects.npz')['arr_0']

    # Fix the input vector
    mygamevector = np.array(docvect.iloc[:,0])
    mygamevector= mygamevector.reshape(-1,1)


    ##### SEMANTICS BASED
    mysimilarities = []
    for t in range(0,allgamedocvects.shape[0]):
        currgamevect = allgamedocvects[t,:]
        currgamevect = currgamevect.reshape(-1,1)
        dum = sklpw.cosine_similarity(currgamevect.T,mygamevector.T)
        mysimilarities.append(dum[0][0])
    mycompletesimlist_df = pd.concat((finalgamelist_df['game_rank'],finalgamelist_df['game_name'],pd.DataFrame({'Similarity':mysimilarities})),axis=1)
    mytop10simlist_df = mycompletesimlist_df.copy()
    mytop10simlist_df.sort_values(by='Similarity',ascending=False)
    mytop10simlist_df = mytop10simlist_df[1:11]


    # SORT Etc
    mycompletesimlist_df = pd.concat((finalgamelist_df['game_rank'],finalgamelist_df['game_name'],pd.DataFrame({'Similarity':mysimilarities})),axis=1)
    mytop10simlist_df = mycompletesimlist_df.copy()
    mytop10simlist_df.sort_values(by='Similarity',ascending=False,inplace=True)
    mytop10simlist_df = mytop10simlist_df[1:11]

    # Create output list
    urllist=[]
    for gamename in mytop10simlist_df['game_name']:
        urllist.append(list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'bgg_url'])[0])
    mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['game_name'],'url':urllist})
    mytop10simlist_df.reset_index(drop=True,inplace=True)
    mytop10simlist_df.index = mytop10simlist_df.index+1

    return mytop10simlist_df


# ROUGH

In [29]:
from gensim.models import Word2Vec
bgg_model = Word2Vec.load('datasources/bgg_w2v_model.bin')
words = list(bgg_model.wv.vocab)

In [31]:
len(words)

14630